In [1]:
from pycoingecko import CoinGeckoAPI
import datetime
import time
import pandas as pd
cg = CoinGeckoAPI()
cg_coins = pd.DataFrame(cg.get_coins_list())


def get_cg_id_for_ticker(ticker, name):
    ticker = ticker.lower()
    targets = cg_coins[cg_coins['symbol'].str.lower() == ticker]
    if len(targets) > 1:
        targets = targets[targets['name'].str.lower().str.contains(name.lower())]
    return targets.iloc[0]['id']


def cg_price_ret(cg_id,date):
    print(cg_id, date)
    cg_hist = cg.get_coin_history_by_id(cg_id, pd.to_datetime(date).strftime('%d-%m-%Y'))
    cg_hist_yest = cg.get_coin_history_by_id(cg_id,  (pd.to_datetime(date) - datetime.timedelta(days=1)).strftime('%d-%m-%Y'))
    if 'market_data' not in cg_hist_yest:
        return None
    return cg_hist['market_data']['current_price']['usd'] / cg_hist_yest['market_data']['current_price']['usd']

In [2]:
listings = pd.read_csv('messarilistings.csv')

In [3]:
cg_id = get_cg_id_for_ticker('UNI','Uniswap')

In [4]:
cg_price_ret(cg_id,'2021-01-01')

uniswap 2021-01-01


1.2464503377820928

In [10]:
listings

,Event,Asset,Ticker,Importance,Status,Category,Subcategory,Last Updated,Event Date,Soft Date,nextday
0,WazirX Listing,Band Protocol,BAND,Low,Completed,Token Listing,Centralized Exchange Listing,2020-05-13,2020-05-12,--,NaN
1,USDT Support,Compound,COMP,Medium,Completed,Token Listing,New Token,2020-04-28,2020-04-25,--,NaN
2,Upbit Listing,Origin Protocol,OGN,Low,Completed,Token Listing,Centralized Exchange Listing,2020-04-13,2020-04-13,--,NaN
3,Upbit Listing,Orchid,OXT,Low,Completed,Token Listing,Centralized Exchange Listing,2020-09-10,2020-09-08,--,NaN
4,Uniswap Listing,UMA,UMA,Medium,Completed,Token Listing,Decentralized Exchange Listing,2020-04-30,2020-04-28,--,NaN
...,...,...,...,...,...,...,...,...,...,...,...
276,Binance Listing,Celo,CELO,Low,Completed,Token Listing,Centralized Exchange Listing,2021-01-04,2021-01-04,--,NaN
277,Binance Listing,Nervos Network,CKB,Low,Completed,Token Listing,Centralized Exchange Listing,2021-01-26,2021-01-25,--,NaN
278,Binance Listing,Perpetual Protocol,PERP,Low,Completed,Token Listing,Centralized Exchange Listing,2021-03-19,2021-03-18,--,NaN
279,Anchorage Custody Support,Keep Network,KEEP,Low,Completed,Token Listing,Custody Platform Listing,2020-06-24,2020-06-24,--,NaN


In [40]:
def nextday(df):
    return cg_price_ret(get_cg_id_for_ticker(df['Ticker'],df['Asset']),df['Event Date'])

In [6]:
listings.drop(listings[listings['Event Date']=='--'].index,axis=0,inplace=True)

In [7]:
listings.reset_index(inplace=True)

In [8]:
del listings['index']

In [25]:
i = 0
while i<280:
    try:
        listings['nextday']=nextday(listings.iloc[i])
    except:
        listings['nextday']= None
        continue
    i+=1
    time.sleep(3)

band-protocol 2020-05-12
compound-coin 2020-04-25
origin-protocol 2020-04-13
orchid-protocol 2020-09-08
uma 2020-04-28
kyber-network 2020-05-03
chiliz 2020-10-04
matic-network 2020-04-09
kadena 2020-06-25
origin-protocol 2020-03-13
origin-protocol 2020-03-25
aragon 2020-08-11
avalanche-2 2020-09-20
solana 2020-09-29
filecoin 2020-12-02
near 2020-12-02
nucypher 2020-12-09
the-graph 2020-12-17
props 2021-01-26
flow 2021-02-22
terra-luna 2021-02-23
chiliz 2021-03-02
alpha-finance 2021-03-03
blockstack 2021-03-03
perpetual-protocol 2021-03-09
enjincoin 2021-03-17
matic-network 2021-03-28
skale 2021-03-28
livepeer 2021-04-02
polkadot 2021-03-30
celo 2020-08-17
blockstack 2021-01-16
blockstack 2021-01-16
avalanche-2 2021-02-02
algorand 2021-02-10
maker 2021-03-29
havven 2021-03-29
bitcoin-cash 2021-02-28
loom-network 2020-03-05
algorand 2020-04-06
origin-protocol 2020-03-11
loom-network 2020-03-05
zilliqa 2020-06-17
nervos-network 2020-09-02
polkadot 2020-08-17
cosmos 2020-08-17
kusama 2020-

In [29]:
listings['nextday']

0      0.948336
1      0.948336
2      0.948336
3      0.948336
4      0.948336
         ...   
276    0.948336
277    0.948336
278    0.948336
279    0.948336
280    0.948336
Name: nextday, Length: 281, dtype: float64